In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from hermes.custom.LRS import LRS
# from hermes.custom.RTP import TrainingPlot
from hermes.custom.FC import FileCallback

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [3]:
model.fit(
    ds_train,
    epochs=1,
    validation_data=ds_test,
#     callbacks = [LRS({2:0.45, 4:0.09 , 6 : 0.01 , 10 : 0.05})]
    callbacks = [FileCallback(save_format="csv")]
)

469/469 [==============================] - 6s 5ms/step - loss: 0.3590 - sparse_categorical_accuracy: 0.9001 - val_loss: 0.1998 - val_sparse_categorical_accuracy: 0.9413
